In [15]:
import pandas as pd
import numpy as np
# Raw Data

def merge_symbol(df):
    df['Ticker'] = df['Symbol'].where(
        df['UnderlyingSymbol'].isnull(), df['UnderlyingSymbol'])
    return df


def get_strategy(df):
    df['Strategy'] = None
    df.loc[df['AssetClass'] == 'STK', 'Strategy'] = 'Stock'

    short_put = ((df['Open/CloseIndicator'] == 'O')
                 & (df['Buy/Sell'] == 'SELL')
                 & (df['Put/Call'] == 'P'))
    short_call = ((df['Open/CloseIndicator'] == 'O')
                  & (df['Buy/Sell'] == 'SELL')
                  & (df['Put/Call'] == 'C'))
    long_put = ((df['Open/CloseIndicator'] == 'O')
                & (df['Buy/Sell'] == 'BUY')
                & (df['Put/Call'] == 'P'))
    long_call = ((df['Open/CloseIndicator'] == 'O')
                 & (df['Buy/Sell'] == 'BUY')
                 & (df['Put/Call'] == 'C'))

    df.loc[short_put, 'Strategy'] = 'Short Put'
    df.loc[short_call,'Strategy']='Short Call'
    df.loc[long_put, 'Strategy'] = 'Long Put'
    df.loc[long_call,'Strategy']='Long Call'

    return df

def drop_stock(df):
    df=df[df.Strategy!='Stock']
    return df

def group_trades(df):
    df['Put/Call']=df['Put/Call'].where(df['AssetClass']=='OPT','S')
    df_grouped=df.groupby(group_columns)
    df['TradeDateImp']=group_trade_date(df_grouped,df)
    df['Quantity']=group_quantity(df_grouped)
    return df

def group_close_price(df):
    df['adjusted_close_volume']=df['Quantity']*df['ClosePrice']
    df['adjusted_close']=df.groupby(group_columns)['adjusted_close_volume'].transform('sum')
    df['adjusted_close']=df['adjusted_close_volume']/df['Quantity']
    
def group_quantity(df_grouped):
    return df_grouped['Quantity'].transform('sum')

def group_trade_date(df_grouped,df):
    tmp_trade_date_max=df_grouped['TradeDate'].transform('max')
    tmp_trade_date_min=df_grouped['TradeDate'].transform('min')
    return tmp_trade_date_max.where(df['Open/CloseIndicator']=='C',tmp_trade_date_min)

def drop_duplicates(df):
    return df.drop_duplicates(group_columns)

def extract_relevant_columns(df,columns=['ClientAccountID','Symbol','Open/CloseIndicator','Ticker','Strategy','Expiry','Strike','Quantity','TradeDate','ClosePrice']):
    return df[columns]

def merge_open_close(df_rel):
    df_close=df_rel.loc[df_rel['Open/CloseIndicator']=='C'].drop(['Open/CloseIndicator','Ticker','Strategy'],axis=1)
    df_open=df_rel.loc[df_rel['Open/CloseIndicator']=='O'].drop(['Open/CloseIndicator',],axis=1)
    df=df_open.merge(df_close,on=['ClientAccountID','Symbol','Expiry','Strike',],suffixes=('_Open','_Close'),how='left')
    return df

## Analysis
def get_capital_usage(df):
    column='Capital'
    df[column]=None
    df[column]=df[column].where(df.Strategy!='Short Put',(df['Strike']-df['ClosePrice_Open'])*df['Quantity_Open']*-100)
    df[column]=df[column].where(df.Strategy!='Long Put',(df['ClosePrice_Open'])*df['Quantity_Open']*100)
    df[column]=df[column].where(df.Strategy!='Short Call',(df['Strike'])*df['Quantity_Open']*-100)
    df[column]=df[column].where(df.Strategy!='Long Call',(df['ClosePrice_Open'])*df['Quantity_Open']*100)
    df[column]=df[column].where(df.Strategy!='Stock',(df['ClosePrice_Open'])*df['Quantity_Open']*100)
    return df

def get_profit(df):
    column='Profit'
    df[column]=None
    df[column]=df[column].where(df.ClosePrice_Close.isnull(),(df['ClosePrice_Close']-df['ClosePrice_Open'])*df['Quantity_Close']*100)
    return df

def get_trade_status(df):
    column='TradeStatus'
    df[column]=np.where(df['Quantity_Close'].isnull(),'OPEN','CLOSE')
    return df

def get_max_reward(df):
    column='MaxReward'
    df[column]=None
    df[column]=df[column].where(df.Strategy!='Short Put',(df['ClosePrice_Open'])*df['Quantity_Open']*-100)
    df[column]=df[column].where(df.Strategy!='Long Put',(df['Strike']-df['ClosePrice_Open'])*df['Quantity_Open']*100)
    df[column]=df[column].where(df.Strategy!='Short Call',(df['ClosePrice_Open'])*df['Quantity_Open']*-100)
    return df
def get_ROC(df):
    column='ROC'
    df[column]=None
    df[column]=df[column].where(df['Quantity_Close'].isnull(),df['Profit']/df['Capital'])
    return df

def split_date(df_col):
    #df_col=df_col.apply(str)
    df_col=pd.to_datetime(df_col,format='%Y%m%d')
    year = df_col.dt.year
    month = df_col.dt.month
    day = df_col.dt.day
    return year,month,day

def split_df_date(df):
    df['open_year'],df['open_month'],df['open_day']=split_date(df['TradeDate_Open'])
    df['close_year'],df['close_month'],df['close_day']=split_date(df['TradeDate_Close'])
    return df

def get_days_in_trade(df):
    df['DaysInTrade']=df['TradeDate_Close']-df['TradeDate_Open']+1
    return df

def get_profit_per_day(df):
    df['ProfitPerDay']=df['Profit']/df['DaysInTrade']
    return df

def sort_by_status(df):
    return df.sort_values(by=['TradeStatus'])


def convert_csv_to_report(path):
    df=pd.read_csv(path)
    df=merge_symbol(df)
    df=get_strategy(df)
    df=drop_stock(df)
    df=group_trades(df)
    df=drop_duplicates(df)
    df=extract_relevant_columns(df)
    df=merge_open_close(df)
    df=get_capital_usage(df)
    df=get_trade_status(df)
    df=get_profit(df)
    df=get_max_reward(df)
    df=get_ROC(df)
    df=split_df_date(df)
    df=get_days_in_trade(df)
    df=get_profit_per_day(df)
    df=sort_by_status(df)
    return df

group_columns=['Symbol','Open/CloseIndicator','Buy/Sell','Put/Call']


df=convert_csv_to_report(path)

df

,ClientAccountID,Symbol,Ticker,Strategy,Expiry,Strike,Quantity_Open,TradeDate_Open,ClosePrice_Open,Quantity_Close,...,MaxReward,ROC,open_year,open_month,open_day,close_year,close_month,close_day,DaysInTrade,ProfitPerDay
0,U7662380,ABEO 210319P00002500,ABEO,Short Put,20210319.0,2.5,-2,20210211,0.3181,2.0,...,63.62,0.098813,2021,2,11,2021.0,3.0,8.0,98.0,0.440000
19,U7662380,HGEN 210319P00020000,HGEN,Short Put,20210319.0,20.0,-1,20210210,6.2203,1.0,...,622.03,-0.057657,2021,2,10,2021.0,3.0,8.0,99.0,-0.802525
25,U7662380,KRMD 210319C00005000,KRMD,Short Call,20210319.0,5.0,-1,20210222,0.3250,1.0,...,32.50,-0.050000,2021,2,22,2021.0,3.0,8.0,87.0,-0.287356
53,U7662380,ZDGE 210416P00007500,ZDGE,Short Put,20210416.0,7.5,-1,20210225,1.2250,1.0,...,122.50,-0.175283,2021,2,25,2021.0,3.0,16.0,92.0,-1.195543
28,U7662380,LIVX 210219P00005000,LIVX,Long Put,20210219.0,5.0,2,20210211,0.7799,-2.0,...,844.02,-0.000000,2021,2,11,2021.0,2.0,11.0,1.0,-0.000000
32,U7662380,NAK 210319P00001000,NAK,Short Put,20210319.0,1.0,-5,20210212,0.3528,5.0,...,176.40,-0.147868,2021,2,12,2021.0,3.0,17.0,106.0,-0.451415
33,U7662380,NMTR 210219P00002500,NMTR,Short Put,20210219.0,2.5,-2,20210208,0.8478,2.0,...,169.56,-0.174192,2021,2,8,2021.0,2.0,19.0,12.0,-4.796667
18,U7662380,GPL 210319P00002500,GPL,Short Put,20210319.0,2.5,-2,20210211,1.6948,2.0,...,338.96,-0.019871,2021,2,11,2021.0,3.0,16.0,106.0,-0.030189
38,U7662380,PLTR 210319P00020000,PLTR,Long Put,20210319.0,20.0,1,20210224,0.4200,-1.0,...,1958.00,1.000000,2021,2,24,2021.0,3.0,19.0,96.0,0.437500
40,U7662380,PLTR 210319P00027000,PLTR,Short Put,20210319.0,27.0,-1,20210217,3.7000,1.0,...,370.00,0.000000,2021,2,17,2021.0,2.0,17.0,1.0,0.000000


In [26]:
import pandas as pd
import gspread
import gspread_dataframe as gd

def get_current_ws():
    gc = gspread.oauth()
    ws=gc.open('My Options Journal').worksheet('data')
    return ws

def get_current_df():
    gc = gspread.oauth()
    ws=gc.open('My Options Journal').worksheet('data')
    df=gd.get_as_dataframe(ws).dropna(how='all').dropna(axis=1,how='all')
    return df


def update_worksheet(df):
    ws=get_current_ws()
    ws.clear()
    gd.set_with_dataframe(ws,df)

update_worksheet(df)

In [22]:
df

,ClientAccountID,Symbol,Ticker,Strategy,Expiry,Strike,Quantity_Open,TradeDate_Open,ClosePrice_Open,Quantity_Close,...,MaxReward,ROC,open_year,open_month,open_day,close_year,close_month,close_day,DaysInTrade,ProfitPerDay
0,U7662380,ABEO 210319P00002500,ABEO,Short Put,20210319.0,2.5,-2.0,20210211.0,0.3181,2.0,...,63.62,0.098813,2021.0,2.0,11.0,2021.0,3.0,8.0,98.0,0.440000
1,U7662380,HGEN 210319P00020000,HGEN,Short Put,20210319.0,20.0,-1.0,20210210.0,6.2203,1.0,...,622.03,-0.057657,2021.0,2.0,10.0,2021.0,3.0,8.0,99.0,-0.802525
2,U7662380,KRMD 210319C00005000,KRMD,Short Call,20210319.0,5.0,-1.0,20210222.0,0.3250,1.0,...,32.50,-0.050000,2021.0,2.0,22.0,2021.0,3.0,8.0,87.0,-0.287356
3,U7662380,ZDGE 210416P00007500,ZDGE,Short Put,20210416.0,7.5,-1.0,20210225.0,1.2250,1.0,...,122.50,-0.175283,2021.0,2.0,25.0,2021.0,3.0,16.0,92.0,-1.195543
4,U7662380,LIVX 210219P00005000,LIVX,Long Put,20210219.0,5.0,2.0,20210211.0,0.7799,-2.0,...,844.02,0.000000,2021.0,2.0,11.0,2021.0,2.0,11.0,1.0,0.000000
5,U7662380,NAK 210319P00001000,NAK,Short Put,20210319.0,1.0,-5.0,20210212.0,0.3528,5.0,...,176.40,-0.147868,2021.0,2.0,12.0,2021.0,3.0,17.0,106.0,-0.451415
6,U7662380,NMTR 210219P00002500,NMTR,Short Put,20210219.0,2.5,-2.0,20210208.0,0.8478,2.0,...,169.56,-0.174192,2021.0,2.0,8.0,2021.0,2.0,19.0,12.0,-4.796667
7,U7662380,GPL 210319P00002500,GPL,Short Put,20210319.0,2.5,-2.0,20210211.0,1.6948,2.0,...,338.96,-0.019871,2021.0,2.0,11.0,2021.0,3.0,16.0,106.0,-0.030189
8,U7662380,PLTR 210319P00020000,PLTR,Long Put,20210319.0,20.0,1.0,20210224.0,0.4200,-1.0,...,1958.00,1.000000,2021.0,2.0,24.0,2021.0,3.0,19.0,96.0,0.437500
9,U7662380,PLTR 210319P00027000,PLTR,Short Put,20210319.0,27.0,-1.0,20210217.0,3.7000,1.0,...,370.00,0.000000,2021.0,2.0,17.0,2021.0,2.0,17.0,1.0,0.000000
